### Parallel Training of EIR and Incidence

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import random
import time
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns

In [2]:
# Set seeds for reproducibility
torch.manual_seed(2)
np.random.seed(2)
random.seed(2)

In [3]:
print("Is CUDA available? ", torch.cuda.is_available())
print("GPU count: ", torch.cuda.device_count())
print("GPU Name: ", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU found")

Is CUDA available?  True
GPU count:  8
GPU Name:  NVIDIA RTX A5000


## Data

In [4]:
from src.preprocessing import process_dataframe

In [5]:
# Load and process data
df = pd.read_csv("data/sim_compendia_train/runs/ANC_Simulation_15000_runs_multi_init_eir.csv", index_col=0).reset_index(drop=True)
#df = process_dataframe(df)
df.head()

,run,t,prev_true,EIR_true,vol_true,tested,positive,prev_2to10,inc_2to10,incall
0,1,30,0.227327,7.614221,0.1,201,49,0.301413,0.002055,0.001404
1,1,60,0.227322,6.890392,0.1,191,43,0.301406,0.001860,0.001270
2,1,90,0.224282,6.486069,0.1,180,36,0.297466,0.001758,0.001200
3,1,120,0.220682,6.549618,0.1,227,58,0.292807,0.001781,0.001216
4,1,150,0.218535,5.842044,0.1,152,41,0.290039,0.001591,0.001086


### Exploratory Data Analysis

In [6]:
#from src.util import plot_subplots_for_runs

In [7]:
#plot_subplots_for_runs(df, num_runs=20)

In [8]:
#Cross Correlation
#from src.util import plot_cross_correlation

In [9]:
#plot_cross_correlation(df, 'EIR_true', ['incall'], single_target='incall')

## Preprocessing - Feature Engineering

In [10]:
# Data preparation
train_data = df[df['run'] <= 14500]
eval_data = df[(df['run'] > 14500) & (df['run'] <= 14800)]
test_data = df[df['run'] > 14800]
cols_to_transform = ["EIR_true", "prev_true", "incall"]
log_transform = lambda x: np.log(x + 1e-8)  # Avoids log(0) errors

train_data_scaled = train_data.copy()
eval_data_scaled = eval_data.copy()
test_data_scaled = test_data.copy()
for col in cols_to_transform:
    train_data_scaled[col] = log_transform(train_data_scaled[col])
    eval_data_scaled[col] = log_transform(eval_data_scaled[col])
    test_data_scaled[col] = log_transform(test_data_scaled[col])

In [11]:
#from src.sequence_creator import create_sequences_in_parallel
from src.sequence_creator import create_sequences_1output, create_sequences_multi_inputs

In [12]:
window_size = 10

#Creating sequences of tensors
X_train_EIR, y_train_EIR = create_sequences_1output(train_data_scaled, window_size)
X_eval_EIR, y_eval_EIR = create_sequences_1output(eval_data_scaled, window_size)
X_test_EIR, y_test_EIR = create_sequences_1output(test_data_scaled, window_size)

X_train_Incidence, y_train_Incidence = create_sequences_multi_inputs(train_data_scaled, window_size, features=['prev_true', 'EIR_true'], target='incall')
X_eval_Incidence, y_eval_Incidence = create_sequences_multi_inputs(eval_data_scaled, window_size, features=['prev_true', 'EIR_true'], target='incall')
X_test_Incidence, y_test_Incidence = create_sequences_multi_inputs(test_data_scaled, window_size, features=['prev_true', 'EIR_true'], target='incall')

In [13]:
train_dataset_EIR = torch.utils.data.TensorDataset(X_train_EIR, y_train_EIR)
eval_dataset_EIR = torch.utils.data.TensorDataset(X_eval_EIR, y_eval_EIR)
test_dataset_EIR = torch.utils.data.TensorDataset(X_test_EIR, y_test_EIR)
train_loader_EIR = torch.utils.data.DataLoader(train_dataset_EIR, batch_size=32, shuffle=True)
eval_loader_EIR = torch.utils.data.DataLoader(eval_dataset_EIR, batch_size=32, shuffle=False)
test_loader_EIR = torch.utils.data.DataLoader(test_dataset_EIR, batch_size=32, shuffle=False)

train_dataset_Incidence = torch.utils.data.TensorDataset(X_train_Incidence, y_train_Incidence)
eval_dataset_Incidence = torch.utils.data.TensorDataset(X_eval_Incidence, y_eval_Incidence)
test_dataset_Incidence = torch.utils.data.TensorDataset(X_test_Incidence, y_test_Incidence)
train_loader_Incidence = torch.utils.data.DataLoader(train_dataset_Incidence, batch_size=32, shuffle=True)
eval_loader_Incidence = torch.utils.data.DataLoader(eval_dataset_Incidence, batch_size=32, shuffle=False)
test_loader_Incidence = torch.utils.data.DataLoader(test_dataset_Incidence, batch_size=32, shuffle=False)

In [14]:
X_train_Incidence.shape

torch.Size([3538000, 21, 2])

In [15]:
y_train_EIR.shape

torch.Size([3538000, 1])

In [16]:
class LSTM_EIR(nn.Module):
    def __init__(self, input_size, architecture):
        super(LSTM_EIR, self).__init__()
        self.lstm_layers = nn.ModuleList()
        for i, hidden_size in enumerate(architecture):
            self.lstm_layers.append(nn.LSTM(input_size if i == 0 else architecture[i - 1], hidden_size, batch_first=True))
        self.fc = nn.Linear(architecture[-1], 1)  # Predicting only EIR
    
    def forward(self, x):
        for lstm in self.lstm_layers:
            x, _ = lstm(x)
        x = self.fc(x[:, -1, :])
        return x

class LSTM_Incidence(nn.Module):
    def __init__(self, input_size, architecture):
        super(LSTM_Incidence, self).__init__()
        self.lstm_layers = nn.ModuleList()
        for i, hidden_size in enumerate(architecture):
            self.lstm_layers.append(nn.LSTM(input_size if i == 0 else architecture[i - 1], hidden_size, batch_first=True))
        self.fc = nn.Linear(architecture[-1], 1)# Predicting only incidence
    
    def forward(self, x):
        for lstm in self.lstm_layers:
            x, _ = lstm(x)
        x = self.fc(x[:, -1, :])
        return x

In [17]:
architectures = {
    #"2_layers": [128, 64],
    "3_layers": [200, 100, 50],
    "4_layers": [256, 128, 64, 32],
    #"5_layers": [300, 200, 100, 50, 25]
}

In [18]:
from src.model_exp import train_models_in_parallel

In [ ]:
results = []
for name, architecture in architectures.items():
    print(f"Training models: {name}")
    model_EIR = LSTM_EIR(input_size=1, architecture=architecture)
    model_Incidence = LSTM_Incidence(input_size=2, architecture=architecture)
    trained_EIR, trained_Incidence, loss_history_EIR, loss_history_Incidence, eval_loss_history_EIR, eval_loss_history_Incidence, _ = train_models_in_parallel(
        model_EIR, model_Incidence,
        train_loader_EIR, train_loader_Incidence,
        eval_loader_EIR, eval_loader_Incidence,
        model_name=name, epochs=25, lr=0.001
    )
    results.append({
        "name": name,
        "model_EIR": trained_EIR,
        "model_Incidence": trained_Incidence,
        "loss_history_EIR": loss_history_EIR,
        "loss_history_Incidence": loss_history_Incidence,
        "eval_loss_history_EIR": eval_loss_history_EIR,
        "eval_loss_history_Incidence": eval_loss_history_Incidence
    })

print("Training complete!")


Training models: 3_layers
Epoch 1/25, EIR Loss: 0.0647, Incidence Loss: 0.14845883, Eval EIR Loss: 0.0318, Eval Incidence Loss: 0.03145280
Epoch 2/25, EIR Loss: 0.0287, Incidence Loss: 0.01675272, Eval EIR Loss: 0.0284, Eval Incidence Loss: 0.02476720
Epoch 3/25, EIR Loss: 0.0244, Incidence Loss: 0.01151854, Eval EIR Loss: 0.0210, Eval Incidence Loss: 0.00966674
Epoch 4/25, EIR Loss: 0.0222, Incidence Loss: 0.00849147, Eval EIR Loss: 0.0237, Eval Incidence Loss: 0.00754984
Epoch 5/25, EIR Loss: 0.0210, Incidence Loss: 0.00706027, Eval EIR Loss: 0.0196, Eval Incidence Loss: 0.00576318
Epoch 6/25, EIR Loss: 0.0203, Incidence Loss: 0.00598247, Eval EIR Loss: 0.0205, Eval Incidence Loss: 0.00563611
Epoch 7/25, EIR Loss: 0.0197, Incidence Loss: 0.00582846, Eval EIR Loss: 0.0195, Eval Incidence Loss: 0.00542092
Epoch 8/25, EIR Loss: 0.0199, Incidence Loss: 0.00501644, Eval EIR Loss: 0.0219, Eval Incidence Loss: 0.00790661
Epoch 9/25, EIR Loss: 0.0191, Incidence Loss: 0.00466827, Eval EIR Los

In [ ]:
def compute_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    return {'MSE': mse, 'MAE': mae, 'RMSE': rmse, 'R2': r2}

def evaluate_model(model, loader, device):
    model.eval()
    preds, targets = [], []

    with torch.no_grad():
        for X, y in loader:
            X, y = X.to(device), y.to(device)
            # Only unsqueeze if needed
            if X.dim() == 2:
                X = X.unsqueeze(-1)
            outputs = model(X)
            preds.append(outputs.cpu().numpy().flatten())
            targets.append(y.cpu().numpy().flatten())

    preds = np.concatenate(preds)
    targets = np.concatenate(targets)
    return compute_metrics(targets, preds)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Collect metrics across all models for both EIR and Incidence
all_metrics = []

for res in results:
    model_name = res["name"]
    
    # EIR metrics
    metrics_train_eir = evaluate_model(res["model_EIR"], train_loader_EIR, device)
    metrics_eval_eir = evaluate_model(res["model_EIR"], eval_loader_EIR, device)
    metrics_test_eir = evaluate_model(res["model_EIR"], test_loader_EIR, device)
    
    # Incidence metrics
    metrics_train_inc = evaluate_model(res["model_Incidence"], train_loader_Incidence, device)
    metrics_eval_inc = evaluate_model(res["model_Incidence"], eval_loader_Incidence, device)
    metrics_test_inc = evaluate_model(res["model_Incidence"], test_loader_Incidence, device)
    
    # Append to list for DataFrame creation
    for dataset_name, metrics, target in [
        ("Train", metrics_train_eir, "EIR"),
        ("Eval", metrics_eval_eir, "EIR"),
        ("Test", metrics_test_eir, "EIR"),
        ("Train", metrics_train_inc, "Incidence"),
        ("Eval", metrics_eval_inc, "Incidence"),
        ("Test", metrics_test_inc, "Incidence"),
    ]:
        all_metrics.append({
            "Architecture": model_name,
            "Dataset": dataset_name,
            "Target": target,
            "MSE": metrics["MSE"],
            "RMSE": metrics["RMSE"],
            "MAE": metrics["MAE"],
            "R2": metrics["R2"]
        })

# Convert to DataFrame for easier plotting
df_metrics = pd.DataFrame(all_metrics)

# Define plot function
def plot_metrics_combined(df_metrics):
    metrics_to_plot = ["MSE", "RMSE", "MAE", "R2"]
    n_metrics = len(metrics_to_plot)
    
    fig, axes = plt.subplots(2, n_metrics, figsize=(5 * n_metrics, 10))
    fig.suptitle("Model Performance Metrics by Architecture and Dataset", fontsize=16, y=1.02)

    for i, metric in enumerate(metrics_to_plot):
        for j, target in enumerate(["EIR", "Incidence"]):
            ax = axes[j, i]
            data = df_metrics[df_metrics["Target"] == target]
            sns.barplot(
                data=data,
                x="Architecture", y=metric, hue="Dataset",
                ax=ax, palette="Set2"
            )
            ax.set_title(f"{target} - {metric}")
            ax.set_xlabel("Architecture")
            ax.set_ylabel(metric)
            ax.legend(title="Dataset")
            ax.grid(True, linestyle='--', alpha=0.6)

    plt.tight_layout()
    plt.show()

# Plot all metrics
plot_metrics_combined(df_metrics)


In [ ]:
sns.set(style="whitegrid", font_scale=1.2)
metrics_to_plot = ['MSE', 'MAE', 'RMSE', 'R2']
palette = sns.color_palette("tab10")

for metric in metrics_to_plot:
    g = sns.catplot(
    data=df_metrics, kind="bar",
    x="Architecture", y=metric, hue="Dataset",
    col="Target", palette=palette,
    height=5, aspect=1, legend_out=False
)
    
    g.set_axis_labels("Architecture", metric)
    g.set_titles("{col_name}")
    g._legend.set_title("Dataset Split")
    
    plt.subplots_adjust(top=0.85)
    g.fig.suptitle(f"{metric} Comparison Across Models (EIR & Incidence)", fontsize=16)
    
    plt.show()
